In [2]:
import requests, zipfile
from datetime import date, timedelta
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType, DateType, TimestampType, FloatType
from pyspark.sql.functions import udf, col, lit, when, trim, month, year
from pyspark import SparkFiles

#### call lib

In [3]:
%run ./work/lib.ipynb

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df8643a0-6340-4f87-9f7c-d427045cbb8b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 287ms :: artifacts dl 8ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runt

#### Parametros

In [4]:
number_exercises = 5
end_year = date.today().year
start_year = end_year - number_exercises + 1

year_list = list(range(start_year, end_year + 1))

#### request cvm

In [5]:
def get_cvm_financial_statement(fs_type: str, year: int, spark=spark):
    """
    financial statements type (fs_type)
    -- BPA - Assets
    -- BPP - Liabilities
    -- DRE - Income Statement
    -- DFC_MI - Indirect Cash Flow
    -- DFC_MD - Direct Cash Flow
    """
    import pandas as pd

    url = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_{year}.zip'
    arquivo_zip = f'itr_cia_aberta_{year}.zip'
    arquivo_csv = f'itr_cia_aberta_{fs_type}_con_{year}.csv'

    download = requests.get(url)

    with open(arquivo_zip, 'wb') as arquivo_cvm:
        arquivo_cvm.write(download.content)

    arquivo_zip = zipfile.ZipFile(arquivo_zip)

    stocks_df = pd.read_csv(arquivo_zip.open(name=arquivo_csv), sep=';', encoding='ISO-8859-1')

    df = spark.createDataFrame(stocks_df)

    return df

In [6]:
stocks_df = \
    spark \
        .read \
        .format('csv') \
        .options(header='True', delimiter=';', encoding='iso-8859-1') \
        .csv('{}/extract/stock_list.csv'.format(LAKE_HOME)) \
        .select(
            trim('CNPJ').alias('cnpj'),
            col('TICKER').alias('ticker')
        )

In [15]:
cash_flow_ind_df = None

for year in year_list:

    if cash_flow_ind_df == None:
        cash_flow_ind_df = get_cvm_financial_statement('DFC_MI', year=year)
    else:
        cash_flow_ind_df = cash_flow_ind_df.unionAll(get_cvm_financial_statement('DFC_MI', year=year))

In [23]:
temp_fact_cashflow = \
    cash_flow_ind_df \
        .where(
            (col('ORDEM_EXERC') == 'ÚLTIMO')
            ) \
        .select(
            trim('CNPJ_CIA').alias('cnpj'),
            col('DT_REFER').cast('date').alias('base_date'),
            col('GRUPO_DFP').alias('financial_statement_type'),
            col('MOEDA').alias('currency'),
            col('ESCALA_MOEDA').alias('scale'),
            col('ORDEM_EXERC').alias('order'),
            col('DT_INI_EXERC').cast('date').alias('start_of_period'),
            col('DT_FIM_EXERC').cast('date').alias('end_of_period'),
            trim('CD_CONTA').alias('id_account'),
            col('DS_CONTA').alias('account_description'),
            col('VL_CONTA').cast('decimal(15, 2)').alias('value'),
            col('ST_CONTA_FIXA').alias('account_status'),
            lit('Indirect Cash Flow').alias('type'),
            lit('YTD').alias('period_type')
        )

fact_cash_flow_indirect = \
    temp_fact_cashflow \
        .join(
            stocks_df,
            on=['cnpj'],
            how='inner'
        )

##### Write to DW

In [24]:
df = fact_cash_flow_indirect

write_to_dw(df, 'fact_cashflow_indirect')